## POS tagging using modified Viterbi

### Data Preparation

In [157]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [158]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [159]:
#Reading the data
nltk_data[:10]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [160]:
# Splitting into train and test
random.seed(1235)
train_set, val_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(val_set))
print(train_set[:40])

3718
196
[[('Dan', 'NOUN'), ('E.', 'NOUN'), ('Nelms', 'NOUN'), (',', '.'), ('Valley', 'NOUN'), ('Federal', 'NOUN'), ("'s", 'PRT'), ('president', 'NOUN'), ('and', 'CONJ'), ('chief', 'ADJ'), ('executive', 'NOUN'), ('officer', 'NOUN'), (',', '.'), ('said', 'VERB'), ('0', 'X'), ('the', 'DET'), ('one-time', 'ADJ'), ('charge', 'NOUN'), ('substantially', 'ADV'), ('eliminates', 'VERB'), ('future', 'ADJ'), ('losses', 'NOUN'), ('associated', 'VERB'), ('*', 'X'), ('with', 'ADP'), ('the', 'DET'), ('unit', 'NOUN'), ('.', '.')], [('An', 'DET'), ('official', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('lead', 'NOUN'), ('underwriter', 'NOUN'), ('declined', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('comment', 'VERB'), ('on', 'ADP'), ('the', 'DET'), ('reason', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('delay', 'NOUN'), (',', '.'), ('but', 'CONJ'), ('market', 'NOUN'), ('participants', 'NOUN'), ('speculated', 'VERB'), ('that', 'ADP'), ('a', 'DET'), ('number', 'NOUN'), ('of', 'ADP'), ('factors', 'NOUN'), (',', 

In [161]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95587

In [162]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Dan',
 'E.',
 'Nelms',
 ',',
 'Valley',
 'Federal',
 "'s",
 'president',
 'and',
 'chief']

In [163]:
# vocabulary
V = set(tokens)
print(len(V))
N_Words=int(len(V))

12079


In [164]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print(len(T))
print(T)

12
{'NOUN', 'ADV', 'PRT', 'NUM', 'CONJ', 'X', 'DET', 'PRON', '.', 'ADJ', 'VERB', 'ADP'}


In [165]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [166]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [167]:
# testing the function
print("\n", "Congress")
print(word_given_tag('Congress', 'CONJ'))
print(word_given_tag('Congress', 'NOUN'))
print(word_given_tag('Congress', 'VERB'), "\n")


 Congress
(0, 2147)
(42, 27437)
(0, 12844) 



In [168]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [169]:
# examples
print(t2_given_t1(t2='DET', t1='NOUN'))
print(t2_given_t1('VERB', 'NOUN'))
print(t2_given_t1('CONJ', 'VERB'))

(351, 27437)
(4029, 27437)
(70, 12844)


In [170]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/(t2_given_t1(t2, t1)[1])

In [171]:
tags_matrix

array([[  2.65116453e-01,   1.72394942e-02,   4.35178764e-02,
          9.62204300e-03,   4.23515700e-02,   2.87203416e-02,
          1.27929440e-02,   4.77457466e-03,   2.40223050e-01,
          1.19546596e-02,   1.46845505e-01,   1.76841497e-01],
       [  3.02526597e-02,   8.01196843e-02,   1.46276597e-02,
          3.09175532e-02,   6.64893631e-03,   2.29388289e-02,
          6.94813803e-02,   1.59574468e-02,   1.35638297e-01,
          1.29654258e-01,   3.44414890e-01,   1.19348407e-01],
       [  2.46872947e-01,   1.05332453e-02,   1.97498361e-03,
          5.92495054e-02,   1.97498361e-03,   1.31665571e-02,
          1.00394994e-01,   1.81040149e-02,   4.41079661e-02,
          8.55826214e-02,   3.96971703e-01,   2.10664906e-02],
       [  3.53827953e-01,   2.95595615e-03,   2.77859885e-02,
          1.83564886e-01,   1.38929943e-02,   2.09872887e-01,
          3.54714761e-03,   1.18238246e-03,   1.17351465e-01,
          3.34023051e-02,   1.80313326e-02,   3.45846899e-02],
    

In [172]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [173]:
tags_df

,NOUN,ADV,PRT,NUM,CONJ,X,DET,PRON,.,ADJ,VERB,ADP
NOUN,0.265116,0.017239,0.043518,0.009622,0.042352,0.028720,0.012793,0.004775,0.240223,0.011955,0.146846,0.176841
ADV,0.030253,0.080120,0.014628,0.030918,0.006649,0.022939,0.069481,0.015957,0.135638,0.129654,0.344415,0.119348
PRT,0.246873,0.010533,0.001975,0.059250,0.001975,0.013167,0.100395,0.018104,0.044108,0.085583,0.396972,0.021066
NUM,0.353828,0.002956,0.027786,0.183565,0.013893,0.209873,0.003547,0.001182,0.117351,0.033402,0.018031,0.034585
CONJ,0.351653,0.054495,0.004658,0.040987,0.000466,0.008850,0.119236,0.058687,0.034932,0.120168,0.154169,0.051700
X,0.061841,0.025727,0.183285,0.002876,0.010227,0.075423,0.055449,0.056408,0.166027,0.017098,0.202621,0.143017
DET,0.637835,0.011718,0.000242,0.022469,0.000483,0.045059,0.005799,0.003382,0.017758,0.206692,0.039502,0.009060
PRON,0.208445,0.033781,0.012668,0.006910,0.004607,0.092514,0.009597,0.008061,0.040307,0.071785,0.488676,0.022649
.,0.222561,0.052589,0.002513,0.080589,0.058063,0.026654,0.173472,0.065871,0.093781,0.044602,0.088307,0.090909
ADJ,0.699327,0.004760,0.010504,0.021008,0.017397,0.021664,0.004924,0.000492,0.064172,0.065813,0.012309,0.077630


### Build the vanilla Viterbi based POS tagger

In [174]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    V=[i[0] for i in train_bag]
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
     
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]            
            state_probability = emission_p * transition_p 
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [175]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(val_set)) for x in range(5)]

# list of sents
val_run = [val_set[i] for i in rndom]

# list of tagged words
val_run_base = [tup for sent in val_run for tup in sent]

# list of untagged words
val_tagged_words = [tup[0] for sent in val_run for tup in sent]
val_tagged_words

['Neither',
 'Equus',
 'nor',
 'Tony',
 'Lama',
 'gave',
 'a',
 'reason',
 'for',
 'the',
 'changed',
 'offer',
 'and',
 'Tony',
 'Lama',
 'could',
 "n't",
 'be',
 'reached',
 '*-1',
 'for',
 'comment',
 '.',
 'He',
 'said',
 '0',
 'Chrysler',
 'fully',
 'expects',
 '*-1',
 'to',
 'have',
 'them',
 'installed',
 '*-2',
 'across',
 'its',
 'light-truck',
 'line',
 'by',
 'the',
 'Sept.',
 '1',
 ',',
 '1991',
 ',',
 'deadline',
 '.',
 'Mr.',
 'Kaminski',
 ',',
 'the',
 'schoolteacher',
 ',',
 'and',
 'William',
 'Mehrens',
 ',',
 'a',
 'Michigan',
 'State',
 'University',
 'education',
 'professor',
 ',',
 'concluded',
 'in',
 'a',
 'study',
 'last',
 'June',
 'that',
 'CAT',
 'test',
 'versions',
 'of',
 'Scoring',
 'High',
 'and',
 'Learning',
 'Materials',
 'should',
 "n't",
 'be',
 'used',
 '*-2',
 'in',
 'the',
 'classroom',
 'because',
 'of',
 'their',
 'similarity',
 'to',
 'the',
 'actual',
 'test',
 '.',
 'But',
 'with',
 'the',
 'index',
 'proving',
 'somewhat',
 'better',
 'th

In [176]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(val_tagged_words)
end = time.time()
difference = end-start

In [177]:
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j] 


In [178]:
#The validation accuracy comes as below which needs to be improved upon
accuracy = len(check)/len(tagged_seq)
accuracy

0.9502762430939227

In [179]:
#Incorrect cases
incorrect_tagged_cases = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]

In [180]:
incorrect_tagged_cases

[[('somewhat', 'ADV'), (('better', 'ADV'), ('better', 'ADJ'))],
 [('anticipated', 'VERB'), (('report', 'VERB'), ('report', 'NOUN'))],
 [('to', 'PRT'), (('arrive', 'NOUN'), ('arrive', 'VERB'))],
 [('prices', 'NOUN'), (('firmed', 'NOUN'), ('firmed', 'VERB'))],
 [('then', 'ADV'), (('faltered', 'NOUN'), ('faltered', 'VERB'))],
 [('Media', 'NOUN'), (('preferred', 'VERB'), ('preferred', 'ADJ'))],
 [('stock', 'NOUN'), (('that', 'ADP'), ('that', 'DET'))],
 [('that', 'DET'), (('*T*-122', 'NOUN'), ('*T*-122', 'X'))],
 [("'s", 'PRT'), (('common', 'ADJ'), ('common', 'NOUN'))]]

### Solve the problem of unknown words

In [181]:
# Viterbi Heuristic
#Modifying the vannila viterbi
def Viterbi_unk(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    V=[i[0] for i in train_bag]
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
     
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]      
            if words[key] in V:
                state_probability = emission_p * transition_p
            else:
                state_probability = transition_p    #Considering only the transition prob as emission will be zero        
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        
        state.append(state_max)      
    return list(zip(words, state))



In [182]:
# tagging the test sentences
start = time.time()
tagged_seq_unk = Viterbi_unk(val_tagged_words)
end = time.time()
difference = end-start

In [183]:
check_unk = [i for i, j in zip(tagged_seq_unk, val_run_base) if i == j] 


### Accuracy after implementing first technique of using only transition probabilities

In [184]:
#Achieved an accuracy increase of 3% on validation set
accuracy = len(check_unk)/len(tagged_seq_unk)
accuracy

0.9558011049723757

In [185]:
incorrect_tagged_cases_unk = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_unk, val_run_base)) if j[0]!=j[1]]

In [186]:
incorrect_tagged_cases_unk

[[('their', 'PRON'), (('similarity', 'VERB'), ('similarity', 'NOUN'))],
 [('somewhat', 'ADV'), (('better', 'ADV'), ('better', 'ADJ'))],
 [('anticipated', 'VERB'), (('report', 'VERB'), ('report', 'NOUN'))],
 [('prices', 'NOUN'), (('firmed', 'NOUN'), ('firmed', 'VERB'))],
 [('Media', 'NOUN'), (('preferred', 'VERB'), ('preferred', 'ADJ'))],
 [('stock', 'NOUN'), (('that', 'ADP'), ('that', 'DET'))],
 [('that', 'DET'), (('*T*-122', 'DET'), ('*T*-122', 'X'))],
 [("'s", 'PRT'), (('common', 'ADJ'), ('common', 'NOUN'))]]

### Writing another function for technique 2 where we use patterns to replace tags for the unknown words

In [188]:
def improvement_func():
    
    patterns = [(r'^[0-9]+(.[0-9]+)?$', 'NUM'),
                       (r'.*ing$', 'VERB'),
                       (r'.*ated', 'VERB'),
                       (r'.*ers$', 'NOUN'),
                       (r'.*ment$', 'NOUN'),
                       (r'.*town$', 'NOUN'), 
                       (r'.*ful$', 'ADJ'),
                       (r'.*ous$', 'ADJ'),
                       (r'.*ble$', 'ADJ'),                        
                       (r'.*','NOUN')]  
    regexp_tagger = nltk.RegexpTagger(patterns)
    # help(regexp_tagger)
    rt=regexp_tagger.tag_sents(val_set[0])
    #print(rt)
    incorrect_words=[i[1][0] for i in incorrect_tagged_cases_unk]
    #print(incorrect_words)
    regex_result=regexp_tagger.tag_sents(incorrect_words)
    incorrect_words
    #regex_result    
    for i in incorrect_words[:]:
        tagged_seq_unk.remove(i)
    for i in regex_result:
        tagged_seq_unk.append(i[0])
    tagged_seq_unk.sort()
    val_run_base.sort()
    check_unk = [i for i, j in zip(tagged_seq_unk, val_run_base) if i == j] 
    accuracy = len(check_unk)/len(tagged_seq_unk)
    print(accuracy)
    
    
  

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [191]:
#The validation accuracy for the vanilla viterbi is  as below 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9502762430939227

In [189]:
#The validation accuracy for the modified code is  as below 
improvement_func()

0.9723756906077348


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications
#### Here incorrect_tag_cases_unknown_mod are the words which are still incorrect after modfication and incorrect_tagged_cases are the words which were incorrect with vanilla viterbi. 

In [192]:
incorrect_tagged_cases

[[('somewhat', 'ADV'), (('better', 'ADV'), ('better', 'ADJ'))],
 [('anticipated', 'VERB'), (('report', 'VERB'), ('report', 'NOUN'))],
 [('to', 'PRT'), (('arrive', 'NOUN'), ('arrive', 'VERB'))],
 [('prices', 'NOUN'), (('firmed', 'NOUN'), ('firmed', 'VERB'))],
 [('then', 'ADV'), (('faltered', 'NOUN'), ('faltered', 'VERB'))],
 [('Media', 'NOUN'), (('preferred', 'VERB'), ('preferred', 'ADJ'))],
 [('stock', 'NOUN'), (('that', 'ADP'), ('that', 'DET'))],
 [('that', 'DET'), (('*T*-122', 'NOUN'), ('*T*-122', 'X'))],
 [("'s", 'PRT'), (('common', 'ADJ'), ('common', 'NOUN'))]]

In [195]:
incorrect_tag_cases_unknown_mod=[[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_unk, val_run_base)) if j[0]!=j[1]]
incorrect_tag_cases_unknown_mod

[[('*T*-1', 'X'), (('*T*-122', 'NOUN'), ('*T*-122', 'X'))],
 [('because', 'ADP'), (('better', 'NOUN'), ('better', 'ADJ'))],
 [('faltered', 'VERB'), (('firmed', 'NOUN'), ('firmed', 'VERB'))],
 [('paying', 'VERB'), (('preferred', 'NOUN'), ('preferred', 'ADJ'))],
 [('that', 'ADP'), (('that', 'NOUN'), ('that', 'DET'))]]